In [1]:
# !pip install boto3
import boto3

In [18]:
# I have preconfigured aksk using AWS CLI
s3 = boto3.resource('s3')

# create a s3 bucket
try:
    s3.create_bucket(Bucket='datacont-chunying', CreateBucketConfiguration = {
        'LocationConstraint': 'us-west-2'}
    ) 
except Exception as e:
    print (e)

In [19]:
bucket = s3.Bucket("datacont-chunying")

# make this bucket publicly readable
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'MS4BXCW4N6D6SA3A',
  'HostId': 'NTzAiBBfqjL2iPQrCRzwc+5txcZ1BK9kyD7Cs024NP1R59op3hdN/Il3IVZB/6OGE1Juo/3T3TU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NTzAiBBfqjL2iPQrCRzwc+5txcZ1BK9kyD7Cs024NP1R59op3hdN/Il3IVZB/6OGE1Juo/3T3TU=',
   'x-amz-request-id': 'MS4BXCW4N6D6SA3A',
   'date': 'Mon, 18 Oct 2021 19:54:25 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [20]:
# upload data file into the bucket and make them publicly readable

data_folder = 'data/'
bucket_name = 'datacont-chunying'

file_name = 'experiments.csv'
experiments = open(data_folder+file_name, 'rb')
s3.Object(bucket_name, file_name).put(Body = experiments)
s3.Object(bucket_name, file_name).Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'C8R5GY85YWA78ZPB',
  'HostId': 'KY0xMGkTNTDdkj4pPSq9wZsSLPBikygmR3TduKepeoKHBwlCE7mJqUX75c0nUqJ9yV+bhzdYfpA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KY0xMGkTNTDdkj4pPSq9wZsSLPBikygmR3TduKepeoKHBwlCE7mJqUX75c0nUqJ9yV+bhzdYfpA=',
   'x-amz-request-id': 'C8R5GY85YWA78ZPB',
   'date': 'Mon, 18 Oct 2021 19:54:34 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [21]:
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

In [22]:
table_name = 'DataTable-chunying'

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'ID',
            'KeyType': 'HASH'
        }
    ],
    ID=[
        {
            'AttributeName': 'ID',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

# Print out some data about the table.
print(table.item_count)

0


In [32]:
import csv

s3_endpoint = "https://"+bucket_name+".s3.us-west-2.amazonaws.com/"

with open(data_folder+file_name, 'r') as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf, None)  # skip the headers
    for item in csvf:
        print(item)
        file_item = item[4]
        body = open(data_folder+file_item, 'rb') 
        s3.Object(bucket_name, file_item).put(Body=body)
        md = s3.Object(bucket_name, file_item).Acl().put(ACL='public-read')
        
        url = s3_endpoint + file_item
        metadata_item = {'ID': item[0], 'Temp': item[1],
        'Conductivity' : item[2], 'Concentration' : item[3], 'URL':url}
        
        try: 
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [37]:
# query data
response = table.get_item(
    Key={
        'ID': '2'
    }
)

In [38]:
item = response['Item']
print(item)

{'Temp': '-2', 'Concentration': '3.4', 'ID': '2', 'Conductivity': '52.1', 'URL': 'https://datacont-chunying.s3.us-west-2.amazonaws.com/exp2.csv'}


In [39]:
response

{'Item': {'Temp': '-2',
  'Concentration': '3.4',
  'ID': '2',
  'Conductivity': '52.1',
  'URL': 'https://datacont-chunying.s3.us-west-2.amazonaws.com/exp2.csv'},
 'ResponseMetadata': {'RequestId': 'DE30SNDJEBLL8G5KLTH96GBF7NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 18 Oct 2021 20:10:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '175',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'DE30SNDJEBLL8G5KLTH96GBF7NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3400203744'},
  'RetryAttempts': 0}}